In [2]:
import torch
import numpy as np
import pyro
import matplotlib.pyplot as plt

C:\Users\flose\anaconda3\envs\ml\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_x = torch.linspace(0, 5, 50)
train_y = train_x**3 + torch.normal(torch.zeros(50), torch.ones(50))

train_x.numpy()
plt.scatter(train_x, train_y)